# Deploying Tokkio

In this notebook, we will walk through the steps to deploy the Tokkio workflow on a Bare-metal machine with manual steps. We will use COTURN service to relay media between the Client and Tokkio Back-end. We will work directly on the application instance where the application will be deployed. At a high level the notebook will perform the following:

- Install & BootStrap Kubernetes Cluster Using NVIDIA Cloud Native Stack
- Setup A TURN Server
- Setup Tokkio Foundational Charts:
    - Install Local Path Provisioner
    - Install Tokkio Observability Helm Chart
- Setup Tokkio Application Chart

We'll be referencing the official [Tokkio documentation](https://docs.nvidia.com/ace/tokkio/5.0.0-beta/deployment/bare-metal-manual-setup.html#bare-metal-manual-setup) for setting up the Tokkio application.

## Import Dependencies

In [300]:
import os

## Set Secrets

Tokkio leverages a number of secrets in order to function properly. At a high-level, the following secrets are needed for the Tokkio deployment:

- `OPENAI_API_KEY`: Key to access OpenAI’s models through the API.
- `NGC_CLI_API_KEY`: Key to access NGC resources thru command line. Such as Helm charts, Models, Container images etc. Please make sure to generate a **Personal Key** in an NGC org that has access to Tokkio 5.0.
- `NVIDIA_API_KEY`: Key to access NGC resources thru command line. Such as Helm charts, Models, Container images etc. This one is specifically used to access NVIDIA NIMs. Please make sure to generate a **Personal Key** in an NGC org that has access to Tokkio 5.0.
- `ELEVENLABS_API_KEY`: Key to access ElevenLabs API. ElevenLabs creates highly realistic AI voices for text-to-speech and voice cloning, used in audio content and accessibility.

Once you've generated API keys for each service, we can set the values below and reference them in our deployment later.

In [301]:
os.environ["NGC_CLI_API_KEY"] = "<YOUR_NVIDIA_API_KEY>"
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_API_KEY>"
os.environ["NVIDIA_API_KEY"] = "<YOUR_NVIDIA_API_KEY>"
os.environ["ELEVENLABS_API_KEY"] = "<YOUR_ELEVEN_LABS_API_KEY>"

## Configure TURN Server Settings

As mentioned in the `deploy_turn_server.ipynb` notebook, in order to interact with our Tokkio Avatar, we'll need to set up a media relay service. We'll be using the COTURN server we configured as the media relay service. Below we provide the following configurations used to create the TURN server:

- `TURNSERVER_USERNAME` - The username used to configure the COTURN server
- `TURNSERVER_PASSWORD` - The password used to configure the COTRUN server
- `TURNSERVER_IP` - The public IP address of the instance the COTURN server is being hosted on.

In [302]:
os.environ["TURNSERVER_USERNAME"] = "<YOUR_TURN_SERVER_USERNAME>"  # Username
os.environ["TURNSERVER_PASSWORD"] = "<YOUR_TURN_SERVER_PASSWORD>"  # Password
os.environ["TURNSERVER_IP"] = "<YOUR_TURN_SERVER_IP>" # IP address TURN server is hosted on

## Install NVIDIA Cloud Native Stack (CNS)

NVIDIA Cloud Native Stack (CNS) is an open-source, reference architecture designed to simplify the deployment and management of GPU-accelerated, cloud-native workloads using Kubernetes. It is specifically tailored for AI, data science, and high-performance computing applications that leverage NVIDIA GPUs. Below is a table of the components CNS provides out of the box:

| Component                    | Purpose/Functionality                                                                 |
|------------------------------|--------------------------------------------------------------------------------------|
| **Operating System**         | Ubuntu (22.04/24.04), RHEL                                                           |
| **Kubernetes**               | Container orchestration platform                                                     |
| **Helm**                     | Kubernetes package manager                                                           |
| **Container Runtime**        | Containerd or CRI-O for managing containers                                          |
| **NVIDIA GPU Operator**      | Automates GPU driver and toolkit installation, GPU lifecycle management              |
| **NVIDIA Network Operator**  | Automates deployment of NVIDIA networking drivers and features                       |
| **NVIDIA Container Toolkit** | Enables GPU support in containers                                                    |
| **NVIDIA Data Center Driver**| Required GPU drivers for Kubernetes nodes                                            |
| **CNI (e.g., Calico)**       | Container networking interface for Kubernetes networking                             |
| **Optional Add-ons**         | MicroK8s, storage solutions, load balancers, monitoring tools, KServe for inference  |
| **Monitoring Tools**         | GPU metrics export (e.g., DCGM-Exporter), integration with Prometheus                |
| **Device Plugins**           | Enable Kubernetes to schedule and manage GPU, RDMA, and SR-IOV resources             |

We'll leverage CNS to set up a GPU enabled Kubernetes cluster that can be used to run the Tokkio App. First we'll start by creating a `repos` directory to store the CNS Github repo in:

In [3]:
%%bash
mkdir -p ../repos

#### Clone CNS Repo

Once we've created the `repos` directory, we can clone the CNS repo. Notice the clone command below; we are cloning a specific tag. Change this according to your needs:

In [4]:
%%bash
# create subdirectory for cloud native stack
mkdir -p ../repos/cloud-native-stack

# clone NVIDIA cloud native stack repo
git clone --branch v24.11.2 --single-branch https://github.com/NVIDIA/cloud-native-stack.git ../repos/cloud-native-stack

# view files in cloned repo
ls -l ../repos/cloud-native-stack

Cloning into '../repos/cloud-native-stack'...
Note: switching to '36cefa2ccd9700c299682ede909175e4ef3252f0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



total 68
-rwxr-xr-x 1 ubuntu ubuntu  2249 May  6 15:34 CONTRIBUTING.md
-rwxr-xr-x 1 ubuntu ubuntu 11356 May  6 15:34 LICENSE
-rwxr-xr-x 1 ubuntu ubuntu  9268 May  6 15:34 README.md
-rw-r--r-- 1 ubuntu ubuntu 25260 May  6 15:34 cns.json
drwxr-xr-x 3 ubuntu ubuntu  4096 May  6 15:34 install-guides
drwxr-xr-x 6 ubuntu ubuntu  4096 May  6 15:34 playbooks
drwxr-xr-x 2 ubuntu ubuntu  4096 May  6 15:34 troubleshooting


#### Configure CNS `hosts` File

Once cloned, we'll need to update the `hosts` file under the `../repos/cloud-native-stack/playbooks` directory. This file will contain the IP address of the application instance and the username and password to access it. Normally the contents of the file would follow if you plan on deploying tokkio on a remote instance:

```
[master]
<application-instance-ip> ansible_ssh_user=<username> ansible_ssh_pass=<password> ansible_ssh_common_args='-o StrictHostKeyChecking=no'
[nodes]
```

In our case, we're deploying the Tokkio application on our localhost, on a single master node (our Brev instance), so the content of our file will look like the following below:

In [5]:
%%bash
cat > ../repos/cloud-native-stack/playbooks/hosts <<EOF
[master]
localhost ansible_connection=local
EOF

Verify the Config:

In [6]:
%%bash
cat ../repos/cloud-native-stack/playbooks/hosts

[master]
localhost ansible_connection=local


#### Configure CNS Version and Enable CNS Driver

In the cell below, we'll configure our CNS version to 14.0:

In [7]:
%%bash
echo "14.0" > ../repos/cloud-native-stack/playbooks/cns_version.yaml

Once the CNS version has been configured, we'll need to edit the `cns_values_14.0.yaml` file in the `../repos/cloud-native-stack/playbooks/` directory with the following updated entry:

```
cns_nvidia_driver: yes
```

This will install the CNS driver. Once the file has been updated, from the root of the `repos` directory in a seperate terminal, please run the following code block to execute the CNS installation script.

```
cd cloud-native-stack/playbooks
bash setup.sh install
```

It's a requirement to run the script within the `playbooks` directory to get the install step to complete successfully. This step will take a while to complete; about 5-10 minutes. However if you see the terminal output hanging at the below steup, you may have to kill the process and restart or start a new terminal session and run the `bash setup.sh install` command again. If an install does not complete, running a `bash setup.sh uninstall` can be used to uninstall CNS before reapplying an install.

Once the setup completes, you should be able to run the command below to get the status of the GPU Operator pods. If everything is either in a `completed` or `running` state the CNS install step completed successfully!

In [24]:
%%bash
kubectl get pods -n nvidia-gpu-operator

NAME                                                              READY   STATUS      RESTARTS   AGE
gpu-feature-discovery-mmldx                                       1/1     Running     0          2m11s
gpu-operator-1746546513-node-feature-discovery-gc-8545dfbblxw6z   1/1     Running     0          2m21s
gpu-operator-1746546513-node-feature-discovery-master-7458xlnf2   1/1     Running     0          2m21s
gpu-operator-1746546513-node-feature-discovery-worker-lvw9x       1/1     Running     0          2m21s
gpu-operator-59bd9fd876-2tx2q                                     1/1     Running     0          2m21s
nvidia-cuda-validator-hcht2                                       0/1     Completed   0          108s
nvidia-dcgm-exporter-6lkdx                                        1/1     Running     0          2m12s
nvidia-device-plugin-daemonset-nctth                              1/1     Running     0          2m12s
nvidia-operator-validator-sdln6                                   1/1     Ru

## Setting Up Tokkio Foundational Charts

In this section, we will be setting up the Foundational Charts. This will include the local path provisioner and the observability stack. The Rancher Local Path Provisioner automatically creates Kubernetes persistent volumes using local storage on each node, simplifying local storage management for cluster workloads. 

The observability stack we'll deploy will leverage Prometheus & Grafana. Prometheus is an open-source monitoring and alerting tool that collects and stores metrics as time-series data, commonly used for monitoring cloud-native environments like Kubernetes. Grafana is an open-source data visualization platform that displays metrics from sources like Prometheus using interactive dashboards, charts, and graphs. 

Let's get started with providing an overrides file for our Local path provisioner deployment:

In [30]:
%%bash
echo 'nodePathMap:
  - node: DEFAULT_PATH_FOR_NON_LISTED_NODES
    paths:
      - /opt/local-path-provisioner
storageClass:
  name: mdx-local-path
'| envsubst > /tmp/local-path-provisioner-values.yml

#### Fetch Helm Repositories for Local Path Provisioner 

Once the overrides file has been created, we can fetch the helm repositories with the local path provisioner chart we want to install, make sure you have internet access in order to successfully pull public charts:

In [31]:
%%bash
helm repo add containeroo https://containeroo.github.io/helm-charts
helm repo update

"containeroo" has been added to your repositories
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "containeroo" chart repository
...Successfully got an update from the "nvidia" chart repository
Update Complete. ⎈Happy Helming!���


#### Install The Local Path Provisioner

Once the chart has been fetched, we can install the Local Path Provisioner using the `/tmp/local-path-provisioner-values.yml` overrides file we configured earlier:

In [32]:
%%bash
helm upgrade --install local-path-provisioner containeroo/local-path-provisioner \
  --namespace platform \
  --create-namespace \
  --version 0.0.32 \
  -f /tmp/local-path-provisioner-values.yml

Release "local-path-provisioner" does not exist. Installing it now.
NAME: local-path-provisioner
LAST DEPLOYED: Tue May  6 15:51:54 2025
NAMESPACE: platform
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You can create a hostpath-backed persistent volume with a persistent volume claim like this:

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: local-path-pvc
spec:
  accessModes:
    - ReadWriteOnce
  storageClassName: mdx-local-path
  resources:
    requests:
      storage: 2Gi


#### Verify Local Path Provisioner

Once installed, we can verify the creation of the `mdx-local-path` storage class using the command below. Ensure that this storageclass is using the `cluster.local/local-path-provisioner` provisioner:

In [33]:
%%bash
kubectl get storageclass

NAME             PROVISIONER                            RECLAIMPOLICY   VOLUMEBINDINGMODE      ALLOWVOLUMEEXPANSION   AGE
mdx-local-path   cluster.local/local-path-provisioner   Delete          WaitForFirstConsumer   true                   3s


If the output returns as expected, the local path provisioner has been configured! Now we can move on to setting up the observabilty chart.

#### Fetch Helm Repositories for Foundational Chart Observability Stack

As mentioned earlier, the observability stack we'll deploy will leverage Prometheus & Grafana. In order to pull these charts, we'll need access to an NGC API key that can be used to pull resources from NGC. We'll leverage the same secret that was set at the beginning of this notebook to facilitate this process:

In [34]:
%%bash
helm repo add nvidia-ace https://helm.ngc.nvidia.com/nvidia/ace --username '$oauthtoken' --password $NGC_CLI_API_KEY
helm repo update

"nvidia-ace" has been added to your repositories
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "containeroo" chart repository
...Successfully got an update from the "nvidia" chart repository
...Successfully got an update from the "nvidia-ace" chart repository
Update Complete. ⎈Happy Helming!⎈


#### Install The Foundational Chart Observability Stack

Once the chart has been fetched, we can install the Observability Stack using the it's default helm values, we specify version `0.0.5-beta` to be installed:

In [35]:
%%bash
helm upgrade --install obs \
  nvidia-ace/ucf-foundational-chart-observability-stack \
  --version 0.0.5-beta \
  --namespace platform \
  --create-namespace

Release "obs" does not exist. Installing it now.
NAME: obs
LAST DEPLOYED: Tue May  6 15:52:06 2025
NAMESPACE: platform
STATUS: deployed
REVISION: 1


#### Verify Observability Stack Is Running

Once installed, we can verify that our observabilty pod stack is running using the command below. This will return all the status of the observability stack pods in addition to the local path provisioner pod that was created as part of the install process earlier. If all the pods are in a running state, we can move on to deploying the Tokkio application.

In [40]:
%%bash
kubectl get pods -n platform

NAME                                                 READY   STATUS    RESTARTS   AGE
local-path-provisioner-6df889f859-w7q2c              1/1     Running   0          2m33s
obs-grafana-6d7444b987-7n84x                         3/3     Running   0          2m11s
obs-kube-prometheus-stack-operator-cd6d489dd-6lbpk   1/1     Running   0          2m11s
obs-kube-state-metrics-56f7b5474-9lfms               1/1     Running   0          2m11s
obs-loki-0                                           1/1     Running   0          2m11s
obs-opentelemetry-collector-75b9675698-b9mqr         1/1     Running   0          2m11s
obs-prometheus-node-exporter-dstrg                   1/1     Running   0          2m11s
obs-promtail-brrds                                   1/1     Running   0          2m11s
obs-tempo-0                                          1/1     Running   0          2m11s
prometheus-obs-kube-prometheus-stack-prometheus-0    2/2     Running   0          117s


## Tokkio Application Chart Setup

In this section, we will be setting up the Tokkio Helm Chart. This will include the creation of the application namespace and the secrets required for the application. Everything that has been done up until this point has been a prerequisite step to enable to the deployment of the Tokkio application! Firstly, we'll get started with creating a namespace for the application - this will be used to manage all the resources tied to the deploment of the Tokkio app helm chart.

#### Create Tokkio App Namespace

In [239]:
%%bash
kubectl create namespace app

namespace/app created


#### Verify Creation Of Tokkio App Namespace

In [240]:
%%bash
kubectl get ns app

NAME   STATUS   AGE
app    Active   1s


#### Create Tokkio Kubernetes Secrets

We previously created environment variables representing the secrets our Tokkio application will leverage at the beginning of this notebook. Kubernetes Secrets securely store sensitive data (like passwords or API keys) and make it available to applications running in Pods, helping protect sensitive information within your cluster. We'll create a kubernetes secret for each variable, as well as a `docker-registry` secret type that can be used to pull images and resources from NGC:

In [241]:
%%bash
# NGC API Key Secret
kubectl create secret generic ngc-api-key-secret \
  --from-literal=NGC_CLI_API_KEY="${NGC_CLI_API_KEY}" -n app

# OpenAI Key Secret
kubectl create secret generic openai-key-secret \
  --from-literal=OPENAI_API_KEY="${OPENAI_API_KEY}" -n app

# NVIDIA API Key Secret
kubectl create secret generic nvidia-api-key-secret \
  --from-literal=NVIDIA_API_KEY="${NVIDIA_API_KEY}" -n app

# NGC Docker Registry Secret
kubectl create secret docker-registry ngc-docker-reg-secret \
  --docker-server=nvcr.io \
  --docker-username='$oauthtoken' \
  --docker-password="${NGC_CLI_API_KEY}" -n app

# ElevenLabs API Key Secret
kubectl create secret generic elevenlabs-api-key-secret \
  --from-literal=ELEVENLABS_API_KEY="${ELEVENLABS_API_KEY}" -n app

secret/ngc-api-key-secret created
secret/openai-key-secret created
secret/nvidia-api-key-secret created
secret/ngc-docker-reg-secret created
secret/elevenlabs-api-key-secret created


#### Verify Creation Of Tokkio App Secrets

We can use the command below to verify creation of all of the secrets our Tokkio Application will leverage:

In [242]:
%%bash
kubectl get secrets -n app

NAME                        TYPE                             DATA   AGE
elevenlabs-api-key-secret   Opaque                           1      2s
ngc-api-key-secret          Opaque                           1      3s
ngc-docker-reg-secret       kubernetes.io/dockerconfigjson   1      2s
nvidia-api-key-secret       Opaque                           1      2s
openai-key-secret           Opaque                           1      3s


#### Generate Tokkio App Overrides File

Here we will generate the override values file. This file will contain the configuration for the Tokkio Helm Chart, and will be used in helm install command. We'll configure our `TURN_SERVER_IP` using the public IP address we obtained earlier. We'll also configure the application to use the COTURN server we configured earlier!

In [303]:
%%bash
cat <<EOF | envsubst > /tmp/ucf-tokkio-audio-video-app.yml
tokkio-app:
  vms:
    configs:
      vst_config.json:
        network:
          twilio_account_sid: ""
          twilio_auth_token: ""
          use_twilio_stun_turn: false
          static_turnurl_list:
          - "${TURNSERVER_USERNAME}:${TURNSERVER_PASSWORD}@${TURNSERVER_IP}:3478"
          use_reverse_proxy: false

  ia-unreal-renderer-microservice:
    configs:
      IAUEMS_SIGNALLING_SERVER_PEER_CONNECTION_OPTIONS: |
        {
          "iceServers": [
            {
              "urls": ["turn:${TURNSERVER_IP}:3478"],
              "username": "${TURNSERVER_USERNAME}",
              "credential": "${TURNSERVER_PASSWORD}"
            }
          ],
          "iceTransportPolicy": "relay"
        }
EOF

#### View Generated Overrides File

Before installing the Tokkio application, we can verify the contents of the override file:

In [304]:
%%bash
cat /tmp/ucf-tokkio-audio-video-app.yml

tokkio-app:
  vms:
    configs:
      vst_config.json:
        network:
          twilio_account_sid: ""
          twilio_auth_token: ""
          use_twilio_stun_turn: false
          static_turnurl_list:
          - "<YOUR_TURN_SERVER_USERNAME>:<YOUR_TURN_SERVER_PASSWORD>@<YOUR_TURN_SERVER_IP>:3478"
          use_reverse_proxy: false

  ia-unreal-renderer-microservice:
    configs:
      IAUEMS_SIGNALLING_SERVER_PEER_CONNECTION_OPTIONS: |
        {
          "iceServers": [
            {
              "urls": ["turn:<YOUR_TURN_SERVER_IP>:3478"],
              "username": "<YOUR_TURN_SERVER_USERNAME>",
              "credential": "<YOUR_TURN_SERVER_PASSWORD>"
            }
          ],
          "iceTransportPolicy": "relay"
        }


#### Install Tokkio App Helm Chart

Now, we will install the Tokkio helm chart using the override values file and the NGC CLI API key. We're pulling and installing the chart that is configured to use 1 tokkio stream in the `tokkio-1stream-with-ui-5.0.0-beta.tgz`. If we wanted to leverage a chart that uses 3 streams for example, we can use the `tokkio-3stream-with-ui-5.0.0-beta.tgz` chart. Please note that this chart would require a setup using 4x GPUs without GPU time-slicing enabled.

In [245]:
%%bash
helm upgrade \
  --install \
  --reset-values \
  --create-namespace \
  --namespace app \
  --username '$oauthtoken' \
  --password "${NGC_CLI_API_KEY}" \
  --values /tmp/ucf-tokkio-audio-video-app.yml \
  tokkio-app \
  https://helm.ngc.nvidia.com/nvidia/ace/charts/tokkio-1stream-with-ui-5.0.0-beta.tgz

Release "tokkio-app" does not exist. Installing it now.
NAME: tokkio-app
LAST DEPLOYED: Tue May  6 18:27:11 2025
NAMESPACE: app
STATUS: deployed
REVISION: 1


#### Verify Tokkio App Pods Are Running

Once installed, we can verify the Tokkio app is up and running using the command below. Please note it could take anywhere from 30-45 minutes for all the pods to enter a running state. Once running, we can access the app from the UI.

In [248]:
%%bash
kubectl get pods -n app

NAME                                                      READY   STATUS    RESTARTS   AGE
a2f-a2f-deployment-7c48f84597-j5qs8                       1/1     Running   0          10m
ace-configurator-deployment-8649bffb8f-r4h5n              1/1     Running   0          10m
ace-controller-ace-controller-deployment-0                1/1     Running   0          10m
ace-controller-sdr-envoy-sdr-deployment-f58d99587-nk256   4/4     Running   0          10m
anim-graph-sdr-envoy-sdr-deployment-86744bb9-mtzhw        4/4     Running   0          9m59s
ia-animation-graph-microservice-deployment-0              1/1     Running   0          10m
ia-unreal-renderer-microservice-deployment-0              3/3     Running   0          10m
redis-redis-755b568f59-hthv5                              1/1     Running   0          10m
redis-timeseries-redis-timeseries-5dc5ccd4cf-qm4cz        1/1     Running   0          10m
riva-speech-57b77f9466-xmcxw                              1/1     Running   0          1

Once all the pods are in a running state, we can access the Tokkio application via the Tokkio UI!

## Accessing The Tokkio UI

Before attemtping to access the UI, please make sure the following ports are exposed either publically, or on networks you plan on accessing the application from:

- Port `30111`
- Port `30180`
- Port `30443`
- Port `32300`

These ports need to be open to ensure the respective Tokkio services can communicate with each other. Once all these ports have been opened up, we're good to go! To launch the UI in your browser, simply navigate to the following endpoint `https://<app-instance-ip>:30111`

Once accessed, click the “Advanced” when prompted about the unsigned certificate. Review the security message and click “Proceed to…” to continue. Allow audio permissions when prompted by the browser, as you will need an active microphone to interact with the avatar. If the avatar comes up you're good to go! If you run into issues accessing the avatar, please refer to the [Tokkio deployment troubleshooting page](https://docs.nvidia.com/ace/tokkio/5.0.0-beta/deployment/troubleshooting.html)

**Note:** The unsigned certificate warning is expected for development environments. Production deployments should use proper SSL certificates.

## Accessing Observability

Tokkio observability stack helps users collect logs, traces, and metrics across the entire system and provides a single pane of glass for data visualization.

The observability stack is an open-source observability stack designed for log aggregation (Loki) and data visualization (Grafana). and distributed tracing (Tempo). It provides a cost-effective and scalable way to monitor, debug, and analyze modern cloud-native applications. Each component serves a specific purpose in the observability pipeline. For more on the observability stack and it's components, refer to the [Tokkio Observabilty](https://docs.nvidia.com/ace/tokkio/5.0.0-beta/observability/observability.html) documentation page.

You can access the Grafana service from the endpoint: `http://<application-instance-ip>:32300/`. By default the following credentials are configured:

- Username: `admin`
- Password: `admin`

Once you login with these credentials, you have the option to set a new password. For usage and how you can explore traces, metrics and other data points, please refer to the [Tokkio Observabilty Usage](https://docs.nvidia.com/ace/tokkio/5.0.0-beta/observability/observability-usage.html) documentation page.

## Accessing The ACE Configurator:

The ACE Configurator based developer workflow is designed to enable seamless debugging and streamlined iterative development for Tokkio Digital Human Agents. For more details regarding the ACE Configurator, please refer to the [ACE Configurator](https://docs.nvidia.com/ace/ace-configurator/1.0/index.html#ace-configurator-overview)

The workflow satisfies the following use cases:

- Modify the agent pipeline during live deployment and instantly view the updated changes.
- Changes remain persistent even after pod reboot.
- No need to rebuild or redeploy the app for changes supported by the ACE Configurator.

We'll cover the ACE Configurator more extensively in the next notebook `customizing_tokkio.ipynb`. This notebook will cover how to get started with customizations by updating the RAG endpoint used for the current Tokkio deployment!